In [8]:
import os
import cv2
from datetime import datetime, timedelta

#--- SETTINGS --------------------------------------------------------------------------------------
# BGR COLORS
CLR_DEFAULT = (0, 255, 0)
CLR_SELECTION = (0, 200, 0)
CLR_CAPTURING = (0, 0, 255)
CLR_SAVING = (255, 255, 255)

# CAPTURE WINDOW
window_name = "Rock-Paper-Scissors"
capture_width = 1280
capture_height = 720

# ROI
default_roi_dim = 10, 10, 300, 300  # x, y, w, h
roi_file_path = "roi_dim.txt"  # store ROI dimensions to file, None to disable

# CAPTURE SETTINGS
# image will be saved as "{folder}/{img_prefix}{img_next_id}.{img_extension}", example: captured/image_17.png.
folder = "captured"
img_prefix = "rock_"
img_extension = "png"
img_next_id = 0  # Starting id for image naming. If existing, the next highest id in the folder will be assigned
frames_batch = 5  # how many frames to capture in once session
capture_delay = 2000  # time in milliseconds to wait between captures
#---------------------------------------------------------------------------------------------------

# Access the webcam
cap = cv2.VideoCapture(0)

# Check if the webcam is opened successfully
if not cap.isOpened():
    print("Error: Unable to open webcam")
    exit()

# Define capture window
cap.set(cv2.CAP_PROP_FRAME_WIDTH, capture_width)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, capture_height)
cv2.namedWindow(window_name)

# Load ROI from file or use default values
try:
    # Load ROI dimensions from file or use default
    with open("roi_dim.txt", "r") as f:
        x, y, w, h = map(int, f.readline().split())
        # reinit ROI if out of bounds
        if x < 0 or y < 0 or w < 0 or h < 0 or x + w > capture_width or y + h > capture_height:
            raise ValueError

except FileNotFoundError or ValueError:
    # use default values
    x, y, w, h = default_roi_dim

# ROI init settings
is_moving_roi = False
start_x, start_y = 0, 0
square_color = CLR_DEFAULT

# Capture init settings
is_capturing = False
time_counter = datetime.now()
image_counter = 0
os.makedirs(folder, exist_ok=True)

# Mouse event callback function
def mouse_callback(event, mouseX, mouseY, flags, param):
    global x, y, w, h, is_moving_roi, start_x, start_y, square_color

    if event == cv2.EVENT_LBUTTONDOWN and not is_capturing:
        # Check if mouse click is inside ROI
        if x <= mouseX <= x + w and y <= mouseY <= y + h:
            is_moving_roi = True
            start_x, start_y = mouseX - x, mouseY - y

    elif event == cv2.EVENT_MOUSEMOVE:
        if is_moving_roi:
            # Update ROI position while dragging
            x = max(0, min(mouseX - start_x, frame.shape[1] - w))
            y = max(0, min(mouseY - start_y, frame.shape[0] - h))

    elif event == cv2.EVENT_LBUTTONUP:
        is_moving_roi = False
        # save ROI dimensions to file
        if roi_file_path:
            try:
                with open("roi_dim.txt", "w") as f:
                    f.write(f"{x} {y} {w} {h}")
            except Exception as e:
                print(f"Unable to save ROI dimensions to file: {e}")

cv2.setMouseCallback(window_name, mouse_callback)

while True:

    # Capture frame-by-frame
    ret, frame = cap.read()

    # Check if the frame is read correctly
    if not ret:
        print("Error: Unable to capture frame")
        break

    # Flip the frame horizontally
    frame = cv2.flip(frame, 1)

    # Get the ROI
    roi = frame[y:y+h, x:x+w]

    # Check for key press events
    key = cv2.waitKey(1) & 0xFF

    if key == ord('c') and not is_moving_roi:
        is_capturing = True  # Enter capturing mode
    elif key == 27:  # Escape key, exit all modes
        is_capturing = False
        is_moving_roi = False
    elif key == ord('q'):
        break  # Quit the program

    if is_capturing:
        if image_counter < frames_batch:
            if datetime.now() - time_counter > timedelta(milliseconds=capture_delay):
                # assign valid image id and name
                image_ids = [int(img.replace(img_prefix, "").replace(f".{img_extension}", "")) for img in os.listdir(folder)
                             if img.startswith(img_prefix) and img.endswith(f".{img_extension}")]
                if img_next_id in image_ids:
                    # assign next id
                    img_next_id = max(image_ids) + 1
                filepath = os.path.join(folder, f"{img_prefix}{img_next_id}.{img_extension}")
                # Save the ROI image
                cv2.imwrite(filepath, roi)
                image_counter += 1
                time_counter = datetime.now()
                square_color = CLR_SAVING
            else:
                square_color = CLR_CAPTURING
        elif image_counter == frames_batch:
            # reinit counter for next capture
            is_capturing = False
            image_counter = 0

    elif is_moving_roi:
        square_color = CLR_SELECTION

    else:
        square_color = CLR_DEFAULT

    # Draw square around the ROI
    cv2.rectangle(frame, (x, y), (x + w, y + h), square_color, 2)

    # Display the frame
    cv2.imshow(window_name, frame)

# Release the webcam and close all windows
cap.release()
cv2.destroyAllWindows()
